In [169]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import get_scorer_names,mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
import pickle as pkl
from sklearn.preprocessing import OneHotEncoder


In [170]:
df=pd.read_csv("eda.csv")
df.drop(["Unnamed: 0.1","Unnamed: 0"],axis=1,inplace=True)

In [171]:
df.columns

Index(['company', 'job title', 'location', 'job description',
       'salary estimate', 'company_size', 'company_type', 'company_sector',
       'company_industry', 'company_founded', 'company_revenue', 'hourly',
       'rating', 'python_yn', 'spark_yn', 'azure_yn', 'aws_yn', 'excel_yn',
       'machine_learning_yn', 'job_simpl', 'seniority', 'description_len',
       'company_age'],
      dtype='object')

In [172]:
features=df[['company_size','company_type','company_revenue','hourly','rating','company_age','python_yn','spark_yn','azure_yn','aws_yn','excel_yn','machine_learning_yn','job_simpl','seniority','description_len']]
features

,company_size,company_type,company_revenue,hourly,rating,company_age,python_yn,spark_yn,azure_yn,aws_yn,excel_yn,machine_learning_yn,job_simpl,seniority,description_len
0,10000+ Employees,Company - Public,$10+ billion (USD),0,4.4,47.0,1,0,0,1,0,1,data scientist,junior,359
1,10000+ Employees,Hospital,$1 to $5 billion (USD),0,4.0,79.0,1,0,0,0,0,1,data scientist,na,267
2,201 to 500 Employees,Company - Private,Unknown / Non-Applicable,0,4.9,6.0,1,0,0,0,0,0,data scientist,Senior,589
3,Unknown,Company - Public,Unknown / Non-Applicable,0,NaN,NaN,0,0,0,0,0,1,data scientist,junior,132
4,1 to 50 Employees,Company - Private,Unknown / Non-Applicable,0,NaN,NaN,1,0,0,1,0,0,data scientist,na,522
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,1 to 50 Employees,Company - Private,Less than $1 million (USD),0,4.2,NaN,1,0,0,0,0,0,data scientist,junior,70
541,5001 to 10000 Employees,Company - Public,Unknown / Non-Applicable,0,4.7,20.0,0,0,0,0,0,1,data scientist,junior,440
542,1001 to 5000 Employees,Company - Public,$1 to $5 billion (USD),0,4.2,13.0,1,1,0,1,0,1,data scientist,Senior,614
543,10000+ Employees,Company - Public,$10+ billion (USD),0,4.1,111.0,1,0,1,1,0,1,data scientist,Senior,655


In [173]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   company_size         494 non-null    object 
 1   company_type         494 non-null    object 
 2   company_revenue      494 non-null    object 
 3   hourly               545 non-null    int64  
 4   rating               449 non-null    float64
 5   company_age          342 non-null    float64
 6   python_yn            545 non-null    int64  
 7   spark_yn             545 non-null    int64  
 8   azure_yn             545 non-null    int64  
 9   aws_yn               545 non-null    int64  
 10  excel_yn             545 non-null    int64  
 11  machine_learning_yn  545 non-null    int64  
 12  job_simpl            545 non-null    object 
 13  seniority            545 non-null    object 
 14  description_len      545 non-null    int64  
dtypes: float64(2), int64(8), object(5)
memor

Fill Null Values

In [174]:
features['company_size'].fillna(features['company_size'].mode()[0],inplace=True)
features['company_type'].fillna(features['company_type'].mode()[0],inplace=True)
features['company_revenue'].fillna(features['company_revenue'].mode()[0],inplace=True)
features['rating'].fillna(features['rating'].mean(),inplace=True)
features['company_age'].fillna(features['company_age'].mean(),inplace=True)
print(features['company_size'].mode()[0],features['company_type'].mode()[0],features['company_revenue'].mode()[0],features['rating'].mean(),features['company_age'].mean())
features.info()

10000+ Employees Company - Public Unknown / Non-Applicable 4.083964365256125 51.271929824561404
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   company_size         545 non-null    object 
 1   company_type         545 non-null    object 
 2   company_revenue      545 non-null    object 
 3   hourly               545 non-null    int64  
 4   rating               545 non-null    float64
 5   company_age          545 non-null    float64
 6   python_yn            545 non-null    int64  
 7   spark_yn             545 non-null    int64  
 8   azure_yn             545 non-null    int64  
 9   aws_yn               545 non-null    int64  
 10  excel_yn             545 non-null    int64  
 11  machine_learning_yn  545 non-null    int64  
 12  job_simpl            545 non-null    object 
 13  seniority            545 non-null    object 

C:\Users\msiik\AppData\Local\Temp\ipykernel_10996\340273909.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['company_size'].fillna(features['company_size'].mode()[0],inplace=True)
C:\Users\msiik\AppData\Local\Temp\ipykernel_10996\340273909.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['company_type'].fillna(features['company_type'].mode()[0],inplace=True)
C:\Users\msiik\AppData\Local\Temp\ipykernel_10996\340273909.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

In [175]:
cat_features=[]
numerical=[]
for i in features.columns:
    if features[i].dtype=="object":
        cat_features.append(i)
    else:
        numerical.append(i)


cat_features    

['company_size', 'company_type', 'company_revenue', 'job_simpl', 'seniority']

In [176]:
ohe= OneHotEncoder(sparse=False,)
ohe.fit(features[cat_features])


OneHotEncoder(sparse=False)

In [177]:
tr=ohe.transform(features[cat_features])
tr.shape

(545, 33)

In [178]:
features_dum=np.concatenate((features[numerical].values,tr),axis=1)

In [179]:
target=df['salary estimate']

features_dum.shape

(545, 43)

In [180]:
x_train,x_test,y_train,y_test=train_test_split(features_dum,target,train_size=0.8)
x_train

array([[ 0.        ,  4.3       , 94.        , ...,  0.        ,
         1.        ,  0.        ],
       [ 0.        ,  4.        , 24.        , ...,  1.        ,
         0.        ,  0.        ],
       [ 1.        ,  4.08396437, 51.27192982, ...,  1.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  4.        , 51.27192982, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.        ,  4.3       , 94.        , ...,  0.        ,
         1.        ,  0.        ],
       [ 0.        ,  3.8       , 28.        , ...,  0.        ,
         0.        ,  1.        ]])

In [181]:
model=LinearRegression()


In [182]:
np.mean(cross_val_score(model,x_train,y_train,scoring='neg_mean_absolute_error'))

-11497.331217977928

In [183]:
model.fit(x_train,y_train)
y_pred=model.predict(x_test)

In [184]:
mean_absolute_error(y_test,y_pred)

8993.044435849073

In [185]:
rf=RandomForestRegressor()
rf.fit(x_train,y_train)

RandomForestRegressor()

In [186]:
y_pred_rf=rf.predict(x_test)

In [187]:
mean_absolute_error(y_test,y_pred_rf)

3816.7551376146794

In [188]:
Params={'n_estimators':range(50,200,10),'max_features':('auto','sqrt','log2')}
gs=GridSearchCV(rf,Params,scoring='neg_mean_absolute_error')
gs.fit(x_train,y_train)

f:\siika\python\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
f:\siika\python\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
f:\siika\python\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesReg

GridSearchCV(estimator=RandomForestRegressor(),
             param_grid={'max_features': ('auto', 'sqrt', 'log2'),
                         'n_estimators': range(50, 200, 10)},
             scoring='neg_mean_absolute_error')

In [189]:
y_pred_gs=gs.predict(x_test)
mean_absolute_error(y_test,y_pred_gs)

3351.442252803262

In [190]:
gs.best_estimator_

RandomForestRegressor(max_features='log2', n_estimators=180)

In [191]:
data_in=[0.0,
4.4,
47.0,
1.0,
0.0,
0.0,
1.0,
0.0,
1.0,
359.0,
0.0,
1.0,
0.0,
0.0,
0.0,
0.0,
0.0,
0.0,
0.0,
0.0,
1.0,
0.0,
0.0,
0.0,
0.0,
0.0,
0.0,
0.0,
0.0,
1.0,
0.0,
0.0,
0.0,
0.0,
0.0,
0.0,
0.0,
0.0,
0.0,
1.0,
0.0,
1.0,
0.0]

In [192]:
import pickle as pkl
import numpy as np
file_name="FlaskAPI/models/Model_file.p"
with open(file_name,'rb') as pickled:
    model=pkl.load(pickled)

